In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
import pywt
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from skimage.transform import resize
import os
from scipy.io import wavfile
from scipy.io.wavfile import read, write
from scipy.signal import resample_poly
import warnings
import logging
warnings.filterwarnings("ignore")
logging.getLogger('tensorflow').setLevel(logging.ERROR)

2024-07-10 13:02:08.655716: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-10 13:02:08.655786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-10 13:02:08.657274: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-10 13:02:08.668155: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Functions

In [2]:
def signal_resample(signal, original_fs, target_fs):
    gcd = np.gcd(original_fs, target_fs)
    up = target_fs // gcd
    down = original_fs // gcd
    resampled_signal=resample_poly(signal, up, down)
    return resampled_signal
    
def create_scalogram(sig,fs):
    scales = np.arange(1, 41)
    coeffs, _ = pywt.cwt(sig, scales, wavelet = 'morl', sampling_period=1/fs)
    f = abs(coeffs)
    f = resize(f, (40, 250), mode = 'constant')
    return f

def normalize(array):
    return (array - array.min())/(array.max() - array.min()) 

# Loading Data

In [3]:
fs=4000
tensor_rec=np.zeros((1,250,40,1))

original_fs, original_recording = wavfile.read('./Sample_Segments/Good.wav')


DUS_Resampled = signal_resample(original_recording.astype('float32'), original_fs, fs) #Resampling the singal to 4 kHz
DUS_Normalized=normalize(DUS_Resampled) #Normalizing the singal
f=create_scalogram(DUS_Normalized,fs) #Creating the scalogram
f=normalize(f) #Normalizing the scalogram
tensor_rec[0,:,:,0]=f.T 

# Quality Evaluation

In [4]:
model_path = './saved_model'
model = tf.keras.models.load_model(model_path)  # loading the saved model

label = np.argmax(model.predict(tensor_rec), axis=1) 

quality_labels = {0: "Good", 1: "Poor", 2: "Interference", 3: "Talking", 4: "Silent"}

# Print the quality labels
Estimated_label = quality_labels[label[0]]
print(f'The Estimated Quality Class is "{Estimated_label}"')

2024-07-10 13:02:13.420441: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11437 MB memory:  -> device: 0, name: Tesla P100-PCIE-12GB, pci bus id: 0000:04:00.0, compute capability: 6.0
2024-07-10 13:02:16.057452: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906


1/1 [==============================] - 2s 2s/step
The Estimated Quality Class is "Good"
